In [ ]:
import os 
import tensorflow as tf
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchmetrics.image.fid import FrechetInceptionDistance

import matplotlib.pyplot as plt

random_seed = 9292
torch.manual_seed(random_seed)
LATENT_DIM = 100
BATCH_SIZE = 64
NUM_EPOCHS = 500
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS = int(os.cpu_count() / 2)
lr = 3e-4
latent_dims = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data_dir = './data'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

training = CIFAR10(data_dir, train=True, transform=transform, download=True)
testing = CIFAR10(data_dir, train=False, transform=transform, download=True)

train_dataloader = DataLoader(training, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(testing, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
fid = FrechetInceptionDistance(features=2048, normalize=True, reset_real_features=False).to(device)

for i, (real_imgs, _) in enumerate(test_dataloader):
    real_imgs = real_imgs.float().cuda()
    fid.update(real_imgs, real=True)

In [ ]:
from utils import *
from models.DCGAN import DCGANDiscriminator, DCGANGenerator

In [ ]:
disc = DCGANDiscriminator().to(device)
gen = DCGANGenerator(LATENT_DIM).to(device)

disc_optim = optim.Adam(disc.parameters(), lr=lr)
gen_optim = optim.Adam(gen.parameters(), lr=lr)

criterion = nn.BCELoss()

DCGAN_results = train_gan(disc, gen, disc_optim, gen_optim, criterion, 'models/DCGAN3', train_dataloader, test_dataloader, fid)

In [ ]:
disc = ConditionalDCGANDiscriminator().to(device)
gen = ConditionalDCGANGenerator(LATENT_DIM).to(device)

disc_optim = optim.Adam(disc.parameters(), lr=1e-4)
gen_optim = optim.Adam(gen.parameters(), lr=4e-4)

criterion = nn.BCELoss()

ConditionalDCGAN_results = train_conditional_gan(disc, gen, disc_optim, gen_optim, criterion, 'models/C-DCGAN_run_8', train_dataloader, test_dataloader, fid)

disc loss:  0.34  gen loss:  4.30  real acc:  0.82  fake_acc:  0.82
disc loss:  0.63  gen loss:  1.34  real acc:  0.68  fake_acc:  0.57
disc loss:  0.62  gen loss:  1.76  real acc:  0.69  fake_acc:  0.70
disc loss:  0.62  gen loss:  1.19  real acc:  0.64  fake_acc:  0.69
disc loss:  0.62  gen loss:  1.39  real acc:  0.67  fake_acc:  0.70
disc loss:  0.61  gen loss:  1.30  real acc:  0.65  fake_acc:  0.69
disc loss:  0.62  gen loss:  1.43  real acc:  0.66  fake_acc:  0.69
disc loss:  0.61  gen loss:  2.02  real acc:  0.71  fake_acc:  0.75
disc loss:  0.56  gen loss:  1.65  real acc:  0.69  fake_acc:  0.79
disc loss:  0.54  gen loss:  1.64  real acc:  0.69  fake_acc:  0.77
disc loss:  0.52  gen loss:  1.72  real acc:  0.72  fake_acc:  0.81
disc loss:  0.50  gen loss:  1.82  real acc:  0.72  fake_acc:  0.84
disc loss:  0.45  gen loss:  2.34  real acc:  0.77  fake_acc:  0.88
disc loss:  0.44  gen loss:  2.30  real acc:  0.78  fake_acc:  0.88
disc loss:  0.40  gen loss:  2.31  real acc:  0.